In [ ]:
# Workflow 

# topic --> generate outline using LLM for topic --> generate blog using LLM for topic and outline ---> final blog

In [3]:
from langgraph.graph import StateGraph, START, END  
from typing import TypedDict
from euriai.langchain import create_chat_model
from dotenv import load_dotenv
import os
load_dotenv()  # Load environment variables from .env file
api_key = os.getenv("key")  # Retrieve the API key from environment variables

In [4]:
chat_model = create_chat_model(
    api_key=api_key,
    model="gpt-4.1-nano",
    temperature=0.7
)

In [5]:
## Define the state

class LLMState(TypedDict):
    topic: str
    outline: str
    blog: str

In [6]:
def getoutline(state: LLMState) -> LLMState:
    topic = state["topic"]
    prompt = f"Generate a detailed outline for a blog post about the following topic: {topic}"
    response = chat_model.invoke(prompt)
    state["outline"] = response.content
    return state

def getblog(state: LLMState) -> LLMState:
    topic = state["topic"]
    outline = state["outline"]
    prompt = f"Write a blog post about the following topic: {topic} using the following outline: {outline}"
    response = chat_model.invoke(prompt)
    state["blog"] = response.content
    return state

In [7]:
## Define the graph
llm_graph = StateGraph(LLMState)

## Add nodes to the graph
llm_graph.add_node("getOutline", getoutline)
llm_graph.add_node("getBlog", getblog)

## Add edges to the graph
llm_graph.add_edge(START, "getOutline")
llm_graph.add_edge("getOutline", "getBlog")
llm_graph.add_edge("getBlog", END)

## compile the graph
workflow = llm_graph.compile()

In [8]:
## execute the workflow
initial_state = {"topic": "AI: Boon or Bane?"}
final_state = workflow.invoke(initial_state)    

In [11]:
print(final_state['outline'])

**Blog Post Outline: AI: Boon or Bane?**

I. Introduction
   A. Hook: Recent advancements in Artificial Intelligence and their growing presence
   B. Purpose: To explore the dual nature of AI—its benefits and potential risks
   C. Thesis statement: While AI offers transformative opportunities, it also raises significant ethical and societal concerns

II. Understanding Artificial Intelligence
   A. Definition and types of AI (Narrow AI, General AI, Superintelligent AI)
   B. Brief history and evolution of AI technology
   C. Current applications in various sectors

III. The Boon: Benefits of AI
   A. Healthcare
      1. Diagnostics and personalized medicine
      2. Robotic surgeries and patient monitoring
   B. Education
      1. Adaptive learning platforms
      2. Accessibility improvements
   C. Industry and Economy
      1. Automation of manufacturing and logistics
      2. Data analysis and decision-making
   D. Daily Life
      1. Virtual assistants and smart devices
      2. Enh

In [12]:
print(final_state['blog'])

AI: Boon or Bane? Navigating the Double-Edged Sword of Artificial Intelligence

**I. Introduction**

In recent years, artificial intelligence (AI) has transitioned from a futuristic concept to a tangible reality, transforming industries and everyday life alike. From voice assistants to autonomous vehicles, AI’s presence is undeniable and rapidly expanding. As these innovations proliferate, it’s crucial to ask: Is AI a boon—bringing unprecedented benefits—or a bane, posing significant risks? This blog delves into the dual nature of AI, exploring how it can both elevate and endanger our society.

**II. Understanding Artificial Intelligence**

*Definition and Types of AI*

Artificial Intelligence refers to machines or software that simulate human intelligence processes. Its classifications include:

- **Narrow AI:** Designed for specific tasks, like facial recognition or language translation.
- **General AI:** Hypothetical systems with human-like cognitive abilities across diverse tasks.
